# Ant Colony Path Finding (Task 1)

> A total overkill: just for fun and experimentation! </br>
> See readme on my thoughts for using ACO for task 2 and 3.

In [1]:
import numpy as np

In [25]:

grid = [
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0],
    [0, 1, 0, 0, 1],
    [0, 0, 0, 0, 0]
]

# Start and end positions
start = (4, 0)  
end = (0, 4)   

# Ant Colony Optimization parameters
n_ants = 10
n_iterations = 100
decay = 0.5

In [33]:

class AntColonyOptimization:
    def __init__(self, grid, start, end, n_ants, n_iterations, decay, alpha=1, beta=1):
        self.grid = np.array(grid)
        self.start = start
        self.end = end
        self.n_ants = n_ants
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha  # pheromone importance
        self.beta = beta    # distance importance
        self.pheromone = np.ones(self.grid.shape) / 10  # Initialize pheromone with a small value
        
        # Set the pheromone to zero on obstacles
        self.pheromone[self.grid == 1] = 0
        
        self.best_path = None
        self.best_path_length = np.inf

    def run(self):
        for _ in range(self.n_iterations):
            all_paths = self.form_paths()
            self.update_pheromone(all_paths)
            self.evaporate_pheromone()
        return self.best_path, self.best_path_length
    
    def form_paths(self):
        all_paths = []
        for _ in range(self.n_ants):
            path = []
            visited = set()
            position = self.start

            while position != self.end:
                path.append(position)
                visited.add(position)
                next_moves = self.possible_moves(position, visited)
                if not next_moves:
                    # No moves possible, go back to start
                    # Better options could be to backtrack 
                    # and here I assume that there is always a valid path, but if not, there is infinite loop
                    path = [self.start]
                    position = self.start
                    visited = {self.start}
                    continue
                next_move = self.pick_move(next_moves)
                position = next_move

            all_paths.append((path, len(path)))
            
            # Check if we found a better path
            # This is the place we can add elevation infos in task 3
            if self.best_path_length > all_paths[-1][1]:
                self.best_path = all_paths[-1][0]
                self.best_path_length = all_paths[-1][1]

        return all_paths

    def possible_moves(self, position, visited):
        moves = []
        for move in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            next_pos = (position[0] + move[0], position[1] + move[1])
            if (
                0 <= next_pos[0] < self.grid.shape[0]
                and 0 <= next_pos[1] < self.grid.shape[1]
                and next_pos not in visited
                and self.grid[next_pos] != 1  # Avoid obstacles
            ):
                moves.append(next_pos)
        return moves

    def pick_move(self, moves):
        pheromones = [self.pheromone[move] for move in moves]
        distances = [self.heuristic(move) for move in moves]
        probabilities = [(pheromone ** self.alpha) * ((1.0 / distance) ** self.beta) for pheromone, distance in zip(pheromones, distances)]
        probabilities = probabilities / np.sum(probabilities)
        move = np.random.choice(len(moves), p=probabilities)
        return moves[move]

    def update_pheromone(self, all_paths):
        for path, length in all_paths:
            for move in path:
                self.pheromone[move] += 1.0 / length

    def evaporate_pheromone(self):
        self.pheromone *= (1 - self.decay)


    def heuristic(self, position):
        return max(abs(position[0] - self.end[0]) + abs(position[1] - self.end[1]), 1)

aco = AntColonyOptimization(grid, start, end, n_ants, n_iterations, decay)

best_path, best_path_length = aco.run()
best_path, best_path_length


([(4, 0), (3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3)], 8)

> boom!